# Amplitude Modulation

In [ ]:
from disiple.signals import AudioSignal, TimeSignal, PowerSpectrum
from disiple.util import nextpow2
import numpy as np
from bokeh.plotting import show
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider, Div

In [ ]:
samplerate = 8000
duration = nextpow2(samplerate) / samplerate # at least 1 second
times = np.arange(0, duration, 1/samplerate)

### Message Signal

In [ ]:
amp_mod = 0.3
freq_mod = 220/duration
samples_mod = amp_mod * np.sin(2*np.pi*freq_mod*times)
mod_signal = AudioSignal(samples_mod, samplerate)

### Carrier Signal

In [ ]:
amp_carr = 0.5
freq_carr = 2000
samples_carr = amp_carr * np.sin(2*np.pi*freq_carr*times)
carr_signal = AudioSignal(samples_carr, samplerate)

### Amplitude Modulation

In [ ]:
mod_idx = amp_mod / amp_carr
samples_shifted_mod = np.maximum(0, 1 + samples_mod / amp_carr)
shifted_mod_signal = TimeSignal(samples_shifted_mod, samplerate)

In [ ]:
samples_am = samples_shifted_mod * samples_carr
am_signal = AudioSignal(samples_am, samplerate)
am_pspec = PowerSpectrum.from_timesignal(am_signal, dB=True)

In [ ]:
desired_env_signal = TimeSignal(samples_mod + amp_carr, samplerate)
true_env_signal = am_signal.envelope()

### Demodulation

In [ ]:
demod_signal = AudioSignal(true_env_signal.data - amp_carr, samplerate)
demod_pspec = PowerSpectrum.from_timesignal(demod_signal)

### Create Figures

In [ ]:
time_range = (0, 6/freq_mod) # show first 6 periods of signal = 6 * 1/freq_mod
dB_range = (-60, -10)
min_freq_mod, max_freq_mod = freq_mod / 2, 2 * freq_mod
freq_range = (freq_carr-max_freq_mod-5, freq_carr+max_freq_mod+5)

mod_fig = mod_signal.plot(title='Message Signal', x_range=time_range, active_inspect=None) 
carr_fig = carr_signal.plot(title='Carrier Signal', x_range=time_range, active_inspect=None, line_color='olive')
shifted_mod_fig = shifted_mod_signal.plot(title='Scaled & Shifted Message Signal', x_range=time_range, active_inspect=None, tools=[])

am_fig = am_signal.plot(title='Amplitude Modulated Signal', x_range=time_range, active_inspect=None, line_color='olive', legend_label='AM Signal')
am_fig.legend.location = 'bottom_right'
am_fig.legend.click_policy = 'hide'
desired_env_signal.plot(fig=am_fig, active_inspect=None, legend_label='Desired Envelope', line_dash='dashed')
true_env_signal.plot(fig=am_fig, active_inspect=None, legend_label='True Envelope', line_color='crimson', line_dash='dashed', line_dash_offset=30)

pspec_fig = am_pspec.plot(title='Power Spectrum of Amplitude Modulated Signal', x_range=freq_range, y_range=dB_range, active_inspect=None, line_color='purple')

demod_fig = demod_signal.plot(title='Demodulated Signal', x_range=time_range, active_inspect=None, line_color='crimson')
demod_pspec_fig = demod_pspec.plot(title='Power Spectrum of Demodulated Signal', y_range=dB_range, active_inspect=None, line_color='crimson')

mod_idx_div = Div(text='Modulation Factor: <b {}>{:.0f}%</b>'.format('style="color:red"' if mod_idx > 1 else '', 100*mod_idx), width=80, height=10)

### Link time axes

In [ ]:
from itertools import product
temporal_figs = {mod_fig, carr_fig, shifted_mod_fig, am_fig, demod_fig}
for fig1, fig2 in product(temporal_figs, temporal_figs):
    fig1.x_range.js_link('start', fig2.x_range, 'start')
    fig1.x_range.js_link('end', fig2.x_range, 'end')

### Display figures

In [ ]:
plot = column(
    gridplot([mod_fig, carr_fig, shifted_mod_fig, am_fig], ncols=2, width=600), 
    row(pspec_fig, mod_idx_div),
    row(demod_fig, demod_pspec_fig, width=600),
)
show(plot)

### Add Interaction

In [ ]:
amp_mod_slider = Slider(start=0, end=0.5, value=amp_mod, step=.01, title='Message Amplitude')
freq_mod_slider = Slider(start=min_freq_mod, end=max_freq_mod, value=freq_mod, step=10, title='Message Frequency')
amp_carr_slider = Slider(start=0.01, end=0.5, value=amp_carr, step=.01, title='Carrier Amplitude')
freq_carr_slider = Slider(start=1000, end=min(4000, samplerate/2-max_freq_mod), value=freq_carr, step=100, title='Carrier Frequency')

callback = CustomJS(args=dict(modSource=mod_fig.renderers[0].data_source,
                              carrSource=carr_fig.renderers[0].data_source,
                              shiftedModSource=shifted_mod_fig.renderers[0].data_source,
                              amSource=am_fig.renderers[0].data_source,
                              desiredEnvSource=am_fig.renderers[1].data_source,
                              trueEnvSource=am_fig.renderers[2].data_source,
                              freqSource=pspec_fig.renderers[0].data_source,
                              demodSource=demod_fig.renderers[0].data_source,
                              demodPspecSource=demod_pspec_fig.renderers[0].data_source,
                              samplerate=samplerate,
                              ampModSlider=amp_mod_slider,
                              freqModSlider=freq_mod_slider,
                              ampCarrSlider=amp_carr_slider,
                              freqCarrSlider=freq_carr_slider,
                              modIdxDiv=mod_idx_div), code="""
    const ampMod = ampModSlider.value;
    const ampCarr = ampCarrSlider.value;
    const freqMod = freqModSlider.value;
    const freqCarr = freqCarrSlider.value;
    const modIdx = ampMod / ampCarr;

    const modSamples = modSource.data.x.map((t) => ampMod * Math.sin(2*Math.PI*freqMod*t));
    modSource.data = {'x': modSource.data.x, 'y': modSamples};
    const carrSamples = carrSource.data.x.map((t) => ampCarr * Math.sin(2*Math.PI*freqCarr*t));
    carrSource.data = {'x': carrSource.data.x, 'y': carrSamples};
    const shiftedModSamples = modSamples.map((t) => Math.max(0, 1 + t / ampCarr));
    shiftedModSource.data = {'x': shiftedModSource.data.x, 'y': shiftedModSamples};
    const amSamples = shiftedModSamples.map((mod, idx) => mod * carrSamples[idx]);
    amSource.data = {'x': amSource.data.x, 'y': amSamples};
    desiredEnvSource.data = {'x': desiredEnvSource.data.x, 'y': modSamples.map((t) => t + ampCarr)};
    const modFactor = 100 * modIdx;
    const style = modIdx > 1 ? 'style="color:red"' : '';
    modIdxDiv.text = `Modulation Factor: <b ${style}>${modFactor.toFixed(0)}%</b>`;

    const freqs = fourier.rfftFreqs(samplerate);
    freqSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.dBSpectrum(amSamples)};
    
    const envSamples = fourier.envelope(amSamples);
    trueEnvSource.data = {'x': trueEnvSource.data.x, 'y': envSamples};
    const demodSamples = envSamples.map((t) => t - ampCarr);
    demodSource.data = {'x': demodSource.data.x, 'y': demodSamples};
    demodPspecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.dBSpectrum(demodSamples)};
""")
amp_mod_slider.js_on_change('value', callback)
freq_mod_slider.js_on_change('value', callback)
amp_carr_slider.js_on_change('value', callback)
freq_carr_slider.js_on_change('value', callback)

freq_carr_slider.js_on_change('value', CustomJS(args=dict(freqRange=pspec_fig.x_range, maxFreqMod=max_freq_mod), code="""
    freqRange.start = cb_obj.value - maxFreqMod - 5;
    freqRange.end = cb_obj.value + maxFreqMod + 5;
"""))

interative_plot = column(
    gridplot([mod_fig, carr_fig, shifted_mod_fig, am_fig], ncols=2, width=600),
    row(pspec_fig, column(amp_mod_slider, freq_mod_slider, amp_carr_slider, freq_carr_slider, mod_idx_div)),
    row(demod_fig, demod_pspec_fig, width=600),
)
show(interative_plot)

In [ ]:
from bokeh.plotting import save
from bokeh.resources import INLINE
template = f"""
{{% block postamble %}}
    <script src="fourier.js"></script>
    <script>const fourier = new Fourier({len(times)});</script>
{{% endblock %}} """
save(interative_plot, filename='am.html', title='Amplitude Modulation', resources=INLINE, template=template)